# SEC Filings

##### Links
* (https://sec-api.io/profile)
* (https://sec-api.io/docs/query-api#request-parameters)
* (https://pypi.org/project/sec-api/)

## Scrape SEC Edgar Filing Database

In [13]:
def get_sec_filing(form,prior_days):

  '''
  Gather all SEC filings based on the form type and release date.
  '''

  !pip install -- sec-api yahoo_fin secedgar
  import requests, pandas, datetime, bs4, secedgar
  from bs4 import BeautifulSoup
  from pandas.tseries.offsets import BDay
  from secedgar.cik_lookup import get_cik_map

  # Variables/Tables
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
  date = datetime.date.today() - BDay(prior_days)
  links,urls, cik_ = [],[], 0
  #ciks = list(get_cik_map()["ticker"].items())
  #ciks = pandas.DataFrame(ciks, columns = ['ticker','cik'])

  # Retrieving forms released
  url = 'https://www.sec.gov/cgi-bin/current?q1=%s&q2=0&q3=%s' %(prior_days,form)
  r = requests.get(url, headers=headers)
  html_obj = BeautifulSoup(str(r.content), 'html.parser')

  # Compiling links to each company page
  i = 0
  for link in html_obj.find_all('a'):
    if i%2 == 0:
      link_obj = link.get('href')
      r = requests.get('https://www.sec.gov/' + link_obj, headers=headers)
      html_obj = BeautifulSoup(str(r.content), 'html.parser')

      # Derive the published date of the filing
      idx = str(html_obj).find('Accepted')
      date_time = str(html_obj)
      date_time = date_time[idx+43:idx+62]
      try:
        date_time = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')
      except:
        pass

      # Compiling links to each form
      for link in html_obj.find_all('a'):
        link_obj = link.get('href')

        try:
          sec_link = 'https://www.sec.gov/' + link_obj
        except:
          pass

        cikk = sec_link[sec_link.find('data') + 5:sec_link.find('data') + 12].replace('/','')
        if sec_link.find(cikk) > 0 and sec_link.find('txt') > 0:

        #  if cikk != cik_:        # avoid tracking company that double files on same day

            # Get Ticker Name
        #    try:
        #      ticker = ciks.ticker[ciks.cik == str(cikk)].iloc[0]
          links.append((cikk,date_time,sec_link)) # ticker
        #    except:
        #      pass

        #    cik_ = cikk
    i += 1

  return links